## Matching previously blocked dataset

Following the instructons you should have checked out the project and started jupyter notebook in the parent folder.
```
uni-sofia-entity-linking-magellan    <= "jupyter notebook" started here
 |- datasets
 |- notebooks
    |- entity_match_electronics.ipynb
```
 

In [53]:
# Import py_entitymatching package
import py_entitymatching as em
import os
import pandas as pd

In [104]:
# Read the CSV files and set 'ID' as the key attribute
A = em.read_csv_metadata("../dataset/amazon_new_features.csv", key='ID')
B = em.read_csv_metadata("../dataset/best_buy_new_features.csv", key='ID')
# Load the pre-labeled data
G = em.read_csv_metadata('../dataset/sample_blocked_500_labeled.csv', 
                         key='_id',
                         ltable=A, rtable=B, 
                         fk_ltable='ltable_ID', fk_rtable='rtable_ID')

print (f"len(G)={len(G)}, labeled={G['label'].value_counts()[1]}")


Metadata file is not present in the given path; proceeding to read the csv file.


len(G)=500, labeled=14


In [105]:
G.head(3)

,Unnamed: 0,_id,ltable_ID,rtable_ID,ltable_Brand,ltable_Name,ltable_Amazon_Price,ltable_Original_Price,ltable_Features,ltable_Parameters,...,ltable_tfidf,rtable_Brand,rtable_Name,rtable_Price,rtable_Description,rtable_Features,rtable_Parameters,rtable_Parameters2,rtable_tfidf,label
0,5433,5433,5,180,Lenovo,Lenovo ThinkPad T430 2344-C4U 14-Inch Notebook,$999.00,NaN,"i5-3320M(3.3GHz). 4GB RAM, 500GB 7200rpm. 14in HD 1366x768, Intel HD Graphics. Camera, DVD. Wind...",T430 2344 C4U 14 i5 3320M 3.3GHz 4GB 500GB 7200rpm. 14in 1366x768 7 64,...,inch t430 lenovo thinkpad 14,Lenovo,Lenovo ThinkPad 14 Laptop Intel Core i5 4GB Memory 500GB Hard Drive Black T440p - 20AN00DEUS,$949.99,"ThinkPad 14&#34; Laptop - Intel Core i5 - 4GB Memory - 500GB Hard Drive, Read customer reviews a...","14 active-matrix TFT-LCD,4th Gen Intel?? Core??? i5-4210M processor,4GB system memory for basic ...",14 i5 4GB 500GB T440p 20AN00DEUS 14 i5 4GB 500GB 14 4th i5 4210M 4GB 500GB 4600 4.72 1.2 4.0 2 3...,i5 4GB 500GB T440p 20AN00DEUS i5 4GB 500GB 4th i5 4210M 4GB 500GB MP3 Power2Go,20an00deus t440p lenovo 500gb i5,0
1,15008,15008,12,643,Asus,"ASUS Zenbook UX305LA 13.3-Inch Laptop (Intel Core i5, 8GB, 256 GB SSD, Titanium Gold) with Windo...",$749.99,$849.99,13.3-Inch Full-HD IPS Anti-Glare Matte Display with an Ultra-wide 170�� Viewing Angle.. Powerful...,UX305LA 13.3 i5 8GB 256 10 13.3 170�� 5th i5 5200U 2.2GHz 2.7GHz 8 256 10 1.2 802.11ac. 3x 802.1...,...,inch asus zenbook ssd gold,Asus,ASUS Computer International UX31LAXH51T,$899.00,Asus ZENBOOK 13.3&#34; Touch-Screen Laptop - 8GB Memory - 256GB Solid State Drive - Silver Alumi...,"4th Gen Intel?? Core??? i5-4200U processor,Intel?? Core??? i5 processor,Intel?? Turbo Boost Tech...",UX31LAXH51T 13.3 8GB 256GB UX31LAXH51T. 4th i5 4200U i5 8GB DDR3L 13.3 256GB 5000 2 3.0 802.11ac...,UX31LAXH51T 8GB 256GB UX31LAXH51T. 4th i5 4200U i5 8GB DDR3L 256GB 802.11ac 1.6GHz 2.6GHz. MP3 P...,ux31laxh51t asus computer international zseries,0
2,47229,47229,34,2024,Dell,Dell Inspiron 13 7000 Series 13.3-Inch Touchscreen Laptop (i7348-5001SLV),$944.97,NaN,"Intel Core i7-5500U Processor (4M Cache, 2.16 GHz). 8 GB DDR3. 500 GB 5400 rpm Hard Drive. 13.3I...",13 7000 13.3 i7348 5001SLV i7 5500U 4M 2.16 8 DDR3. 500 5400 13.3Inch 8.1; 10,...,inch 7000 13 series dell,Dell,Dell 14 Latitude Notebook 4 GB Memory and 128 GB Solid State Drive Black E6420,$299.99,"14 Latitude Notebook - 4 GB Memory and 128 GB Solid State Drive, Read customer reviews and buy o...","Processor Manufacturer: Intel,Processor Type: Core i7,Processor Speed: 2.70 GHz,Standard Memory:...",14 4 128 E6420 14 4 128 i7 2.70 4 DDR3 128 14 DDR3 4 7 64 4 1.3 10,E6420 i7 DDR3 DDR3,e6420 128 dell gb latitude,0


## Splitting the labeled data into development and evaluation set


In this step, we split the labeled data into two sets: development (I) and evaluation (J). Specifically, the development set is used to come up with the best learning-based matcher and the evaluation set used to evaluate the selected matcher on unseen data.

In [134]:
# Split S into development set (I) and evaluation set (J)
IJ = em.split_train_test(G, train_proportion=0.6, random_state=0)
I = IJ['train']
J = IJ['test']

In [135]:
print(f"I[label==1] = {len(I[I['label'] == 1])}")
print(f"J[label==1] = {len(J[J['label'] == 1])}")

J.head(3)

I[label==1] = 8
J[label==1] = 6


,Unnamed: 0,_id,ltable_ID,rtable_ID,ltable_Brand,ltable_Name,ltable_Amazon_Price,ltable_Original_Price,ltable_Features,ltable_Parameters,...,ltable_tfidf,rtable_Brand,rtable_Name,rtable_Price,rtable_Description,rtable_Features,rtable_Parameters,rtable_Parameters2,rtable_tfidf,label
90,469968,469968,350,71,Asus,Asus 11.6\ Laptop 2GB 32GB | X205TA-UH01-BK,$191.45,NaN,Screen Size: 11.6\ . Screen Resolution: 1366 x 768. Processor Type: Intel. Processor Model: Atom...,11.6 2GB 32GB X205TA UH01 11.6 1366 768. Z3735F. 1.33GHz,...,uh01 bk x205ta asus 32gb,Lenovo,Lenovo Flex 3 2in1 11.6 TouchScreen Laptop Intel Celeron 2GB Memory 32GB eMMC Flash Memory Black...,$279.99,Flex 3 2-in-1 11.6&#34; Touch-Screen Laptop - Intel Celeron - 2GB Memory - 32GB eMMC Flash Memor...,"Windows 10 operating system,11.6 touch screen for hands-on control,Intel?? Celeron?? processor N...",3 2in1 11.6 2GB 32GB 80LY0009US 3 2 1 11.6 2GB 32GB 10 11.6 N3050 2GB 32GB 360?? 3.06 0.86 4.0 1...,2in1 2GB 32GB 80LY0009US 2GB 32GB N3050 2GB 32GB MP3 3x,80ly0009us flex emmc lenovo 32gb,0
254,1504406,1504406,1057,36,Dell,"Dell Inspiron 5000 series 15.6-Inch Laptop - Intel Core Processor, 8GB Memory, 1TB Hard Drive, W...",$832.00,NaN,Intel Core i7-5500U 2.4 GHz Processor (2.4 MB Cache). 8 GB DDR3 RAM. 1 TB 5400 rpm Hard Drive. 1...,5000 15.6 8GB 1TB 8.1 i7 5500U 2.4 2.4 8 DDR3 1 5400 15.6Inch 8.1; 10,...,inch processor series windows dell,Dell,Dell Latitude 14.1 Refurbished Laptop Intel Core i5 8GB Memory 500GB Hard Drive Black E6410-8GB-...,$362.99,"Latitude 14.1&#34; Refurbished Laptop - Intel Core i5 - 8GB Memory - 500GB Hard Drive, Read cust...","Microsoft Windows 7 operating system preinstalled,Intel?? Core??? i5 processor,Intel?? Core??? i...",14.1 i5 8GB 500GB E6410 8GB 500GB W7P 14.1 i5 8GB 500GB 7 i5 i5 8GB DDR3 PC3 14.1 500GB 5400 802...,i5 8GB 500GB E6410 8GB 500GB W7P i5 8GB 500GB i5 i5 8GB DDR3 PC3 500GB 802.11b 3MB 2.4GHz window...,500gb 8gb w7p e6410 dell,0
283,1785465,1785465,1241,386,Lenovo,Lenovo IdeaPad G510s 15.6\ Notebook PC - Intel Core i5-4200M 2.5GHz 6GB 1TB Windows 8.1 (Certif...,$379.99,$789.99,This Certified Refurbished product is tested &amp; certified by Lenovo to look and work like new...,G510s 15.6 i5 4200M 2.5GHz 6GB 1TB 8.1 90 i5 4200M 2.5GHz 3.1GHz 6GB DDR3 1TB 15.6 1366 768 8.1 64,...,5ghz ideapad pc certified lenovo,Lenovo,Lenovo ThinkPad Yoga 2in1 14 TouchScreen Laptop Intel Core i5 8GB Memory 1TB+16GB Hybrid Hard Dr...,$899.99,ThinkPad Yoga 2-in-1 14 Touch-Screen Laptop - Intel Core i5 - 8GB Memory - 1TB+16GB Hybrid Hard ...,"Microsoft Windows 8.1 operating system preinstalled,5th Gen Intel?? Core??? i5-5200U processor,I...",2in1 14 i5 8GB 1TB+16GB 14 20DM008UUS 2 1 14 i5 8GB 1TB+16GB 8.1 5th i5 5200U i5 8GB DDR3L 14 10...,2in1 i5 8GB 1TB+16GB 20DM008UUS i5 8GB 1TB+16GB 5th i5 5200U i5 8GB DDR3L 1TB 940M 802.11ac 3MB ...,yoga 20dm008uus thinkpad hybrid lenovo,0


## Selecting the best learning-based matcher 

Selecting the best learning-based matcher typically involves the following steps:

1. Creating a set of learning-based matchers
2. Creating features
3. Converting the development set into feature vectors
4. Selecting the best learning-based matcher using k-fold cross validation


In [136]:
# Create a set of ML-matchers
dt = em.DTMatcher(name='DecisionTree', random_state=0)
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')

### Creating Features

Next, we need to create a set of features for the development set. *py_entitymatching* provides a way to automatically generate features based on the attributes in the input tables. For the purposes of this guide, we use the automatically generated features.

In [137]:
# Generate features
feature_table = em.get_features_for_matching(A, B, validate_inferred_attr_types=False)

In [138]:
feature_table

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated
0,ID_ID_exm,ID,ID,None,None,exact_match,<function ID_ID_exm at 0x7fe903447af0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
1,ID_ID_anm,ID,ID,None,None,abs_norm,<function ID_ID_anm at 0x7fe90343d160>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
2,ID_ID_lev_dist,ID,ID,None,None,lev_dist,<function ID_ID_lev_dist at 0x7fe903a1edc0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
3,ID_ID_lev_sim,ID,ID,None,None,lev_sim,<function ID_ID_lev_sim at 0x7fe903454160>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
4,Brand_Brand_jac_qgm_3_qgm_3,Brand,Brand,qgm_3,qgm_3,jaccard,<function Brand_Brand_jac_qgm_3_qgm_3 at 0x7fe96cb3af70>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
5,Brand_Brand_cos_dlm_dc0_dlm_dc0,Brand,Brand,dlm_dc0,dlm_dc0,cosine,<function Brand_Brand_cos_dlm_dc0_dlm_dc0 at 0x7fe96cb3aee0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
6,Brand_Brand_jac_dlm_dc0_dlm_dc0,Brand,Brand,dlm_dc0,dlm_dc0,jaccard,<function Brand_Brand_jac_dlm_dc0_dlm_dc0 at 0x7fe96cb3ae50>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
7,Brand_Brand_mel,Brand,Brand,None,None,monge_elkan,<function Brand_Brand_mel at 0x7fe96cb1d550>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
8,Brand_Brand_lev_dist,Brand,Brand,None,None,lev_dist,<function Brand_Brand_lev_dist at 0x7fe96cb320d0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
9,Brand_Brand_lev_sim,Brand,Brand,None,None,lev_sim,<function Brand_Brand_lev_sim at 0x7fe96cb32ee0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True


In [139]:
# List the names of the features generated
feature_table['feature_name']

0                                       ID_ID_exm
1                                       ID_ID_anm
2                                  ID_ID_lev_dist
3                                   ID_ID_lev_sim
4                     Brand_Brand_jac_qgm_3_qgm_3
5                 Brand_Brand_cos_dlm_dc0_dlm_dc0
6                 Brand_Brand_jac_dlm_dc0_dlm_dc0
7                                 Brand_Brand_mel
8                            Brand_Brand_lev_dist
9                             Brand_Brand_lev_sim
10                                Brand_Brand_nmw
11                                 Brand_Brand_sw
12              Features_Features_jac_qgm_3_qgm_3
13          Features_Features_cos_dlm_dc0_dlm_dc0
14          Parameters_Parameters_jac_qgm_3_qgm_3
15      Parameters_Parameters_cos_dlm_dc0_dlm_dc0
16        Parameters2_Parameters2_jac_qgm_3_qgm_3
17    Parameters2_Parameters2_cos_dlm_dc0_dlm_dc0
18                    tfidf_tfidf_jac_qgm_3_qgm_3
19                tfidf_tfidf_cos_dlm_dc0_dlm_dc0


### Converting the Development Set to  Feature Vectors

In [140]:
# Convert the I into a set of feature vectors using F
H = em.extract_feature_vecs(I, 
                            feature_table=feature_table, 
                            attrs_after='label',
                            show_progress=True) 

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [141]:
# Display first few rows
H.head(3)

,_id,ltable_ID,rtable_ID,ID_ID_exm,ID_ID_anm,ID_ID_lev_dist,ID_ID_lev_sim,Brand_Brand_jac_qgm_3_qgm_3,Brand_Brand_cos_dlm_dc0_dlm_dc0,Brand_Brand_jac_dlm_dc0_dlm_dc0,...,Parameters2_Parameters2_cos_dlm_dc0_dlm_dc0,tfidf_tfidf_jac_qgm_3_qgm_3,tfidf_tfidf_cos_dlm_dc0_dlm_dc0,tfidf_tfidf_jac_dlm_dc0_dlm_dc0,tfidf_tfidf_mel,tfidf_tfidf_lev_dist,tfidf_tfidf_lev_sim,tfidf_tfidf_nmw,tfidf_tfidf_sw,label
145,700493,532,1932,0,0.275362,2,0.50,1.0,1.0,1.0,...,0.223607,0.066667,0.2,0.111111,0.531028,30,0.117647,-4.0,6.0,0
158,744037,571,1836,0,0.311002,4,0.00,1.0,1.0,1.0,...,0.204124,0.019231,0.2,0.111111,0.523477,26,0.161290,-6.0,4.0,0
150,713219,542,1525,0,0.355410,3,0.25,1.0,1.0,1.0,...,0.246183,0.058824,0.2,0.111111,0.509740,23,0.178571,-1.0,6.0,0


### Selecting the Best Matcher Using Cross-validation

Now, we select the best matcher using k-fold cross-validation. For the purposes of this guide, we use five fold cross validation and use 'precision' and 'recall' metric to select the best matcher.

In [142]:
# Select the best ML matcher using CV
result = em.select_matcher([dt, rf, svm, ln, lg], table=H, 
        exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'],
        k=5,
        target_attr='label', metric_to_select_matcher='precision', random_state=0)
result['cv_stats']

/home/bobi/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/bobi/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/bobi/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/bobi/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedM

/home/bobi/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/bobi/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regressi

,Matcher,Average precision,Average recall,Average f1
0,DecisionTree,0.0,0.0,0.0
1,RF,0.0,0.0,0.0
2,SVM,0.0,0.0,0.0
3,LinReg,0.0,0.0,0.0
4,LogReg,0.0,0.0,0.0


In [144]:
# Select the best ML matcher using CV
result = em.select_matcher([dt, rf, svm, ln, lg], table=H, 
        exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'],
        k=5,
        target_attr='label', metric_to_select_matcher='recall', random_state=42)
result['cv_stats']

/home/bobi/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/bobi/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/bobi/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/bobi/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedM

/home/bobi/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/bobi/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regressi

,Matcher,Average precision,Average recall,Average f1
0,DecisionTree,0.05,0.1,0.066667
1,RF,0.00,0.0,0.000000
2,SVM,0.00,0.0,0.000000
3,LinReg,0.00,0.0,0.000000
4,LogReg,0.00,0.0,0.000000


### Converting the Evaluation Set to  Feature Vectors

As before, we convert to the feature vectors (using the feature table and the evaluation set)

In [145]:
# Convert J into a set of feature vectors using feature table
L = em.extract_feature_vecs(J, feature_table=feature_table,
                            attrs_after='label', show_progress=True)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


### Training the Selected Matcher

In [146]:
len(L)

200

In [152]:
# Train using feature vectors from I 
rf.fit(table=H, 
       exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], 
       target_attr='label')

### Predicting the Matches

Next, we predict the matches for the evaluation set (using the feature vectors extracted from it).

In [153]:
# Predict on L 
predictions = rf.predict(table=L, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], 
              append=True, target_attr='predicted', inplace=False)

In [154]:
predictions[predictions['predicted']==1]

,_id,ltable_ID,rtable_ID,ID_ID_exm,ID_ID_anm,ID_ID_lev_dist,ID_ID_lev_sim,Brand_Brand_jac_qgm_3_qgm_3,Brand_Brand_cos_dlm_dc0_dlm_dc0,Brand_Brand_jac_dlm_dc0_dlm_dc0,...,tfidf_tfidf_jac_qgm_3_qgm_3,tfidf_tfidf_cos_dlm_dc0_dlm_dc0,tfidf_tfidf_jac_dlm_dc0_dlm_dc0,tfidf_tfidf_mel,tfidf_tfidf_lev_dist,tfidf_tfidf_lev_sim,tfidf_tfidf_nmw,tfidf_tfidf_sw,label,predicted
118,559975,423,743,0,0.569314,2,0.333333,1.0,1.0,1.0,...,0.352941,0.4,0.25,0.714646,15,0.545455,14.0,20.0,0,1
463,5984355,3721,548,0,0.147272,4,0.000000,1.0,1.0,1.0,...,0.220339,0.4,0.25,0.649355,31,0.205128,-3.0,12.0,0,1


### Evaluating the Matching Output

Finally, we evaluate the accuracy of predicted outputs

In [155]:
# Evaluate the predictions
eval_result = em.eval_matches(predictions, 'label', 'predicted')
em.print_eval_summary(eval_result)

Precision : 0.0% (0/2)
Recall : 0.0% (0/6)
F1 : 0.0%
False positives : 2 (out of 2 positive predictions)
False negatives : 6 (out of 198 negative predictions)
